In [1]:
import "fmt"
import "io"
import "os"
import "bytes"
import "time"

#### Interface assignability

In [2]:
var w io.Writer // Writer is an interface
w = os.Stdout // os.File has write method
w = new(bytes.Buffer) 
 // w = time.Second // compilation failure

//cannot convert typed constant 1s <time.Duration> to <io.Writer>
//	reason: time.Duration does not implement io.Writer: missing method 
// func (interface{Write(p []uint8) (n int, err error)}).Write(p []uint8) (n int, err error)

In [3]:
type Point struct{X,Y int}

type Circle struct{
    Center Point
    Radius int
}

type Square struct{
    Side int
}

func (circle *Circle) String() string{
    return fmt.Sprintf("Circle at %v with radius %d",circle.Center,circle.Radius)
}

func (square Square) String() string{
    return fmt.Sprintf("Square with side %d",square.Side)
}

var stringer fmt.Stringer 

circle := Circle{Point{3,4},6}

// stringer = &circle // Need &circle because the receiver type in String() is *Circle

// fmt.Println(stringer.String()) // This works in GoLand but not here!!

stringer = Square{4}

fmt.Println(stringer.String()) // Can only call the interface methods

Square with side 4


19 <nil>

#### Empty interface 

In [4]:
var emptyIFace interface{} // This is like Object type in java
fmt.Printf("%T %v\n",emptyIFace,emptyIFace) // The type is nil as well as the value

emptyIFace = 45
fmt.Printf("%T %v\n",emptyIFace,emptyIFace)

emptyIFace = 45.56
fmt.Printf("%T %v\n",emptyIFace,emptyIFace)

emptyIFace = []int{4,5,6}
fmt.Printf("%T %v\n",emptyIFace,emptyIFace)

emptyIFace = Circle{Point{1,2},6}
fmt.Printf("%T %v\n",emptyIFace,emptyIFace)

emptyIFace = nil
fmt.Printf("%T %v\n",emptyIFace,emptyIFace) // Again type is nil as well as the value


<nil> <nil>
int 45
float64 45.56
[]int [4 5 6]
struct { Center struct { X int; Y int }; Radius int } {{1 2} 6}
<nil> <nil>


12 <nil>

#### interface value comparision

comparing interfaces with uncomparable types causes panic

In [5]:
var iface1 interface{}
var iface2 interface{}

fmt.Println(iface1==iface2) // both are nil

iface1 = Point{2,3}
iface2 = Point{3,4}

fmt.Println(iface1==iface2) // false: different values

iface1 = []int{1,2,3}
iface2 = []int{3,4,5}

//fmt.Println(iface1==iface2) // comparing uncomparable type []int

true
false


#### nil reference inside a non nil interface

<a href="L_06_Interfaces/InterfaceNilReference.go">Nil reference inside interface</a>

In [6]:
// Try this in GoLand

type Person struct{
    Name string
    Age int
}

func (person *Person) String() string{
    return fmt.Sprintf("%s has age %d",person.Name,person.Age)
}

var person *Person

fmt.Printf("%T %v\n",person,person) // *main.Person <nil>

var personInfo fmt.Stringer

personInfo = person // NOT to ever assign nil reference to an interface, better use reference in first place

fmt.Printf("%T %v\n",personInfo,personInfo) // *main.Person <nil>

if person != nil { // This check works
    fmt.Println(person.String())
}

fmt.Println("Trying with interface")

if personInfo != nil { // This check doesn't work, the personInfo is not nil but the reference to person it contains is nil
    fmt.Println(personInfo.String())
}

// panic: runtime error: invalid memory address or nil pointer dereference


ERROR: repl.go:18:1: error compiling assignment: personInfo = person
	repl.go:18:1: cannot convert type <*main.Person> to interface <fmt.Stringer>: missing method  String

#### Interface declaration and satisfaction

In [7]:
type Shape interface {
	Area() float64
}

type Line struct{}

// Interface satisfied by an empty struct
func (line Line) Area() float64 {
	return 0.0
}

type Circle float64

// Interface satisfied by a float64
func (radius Circle) Area() float64 {
	return float64(radius * radius * (22.0 / 7.0))
}

type Triangle struct {
	Base   float64
	Height float64
}

// Interface satisfied by a triangle
func (t Triangle) Area() float64 {
	return t.Base * t.Height * 0.5
}

shapes := []Shape{Line{}, Circle(7), Triangle{3, 6}}

for _, shape := range shapes {
    fmt.Printf("Area of %T is %v\n", shape, shape.Area())
}




ERROR: reflect.Value.Convert: value of type float64 cannot be converted to type *struct { 𒀪 xreflect.InterfaceHeader; Area func() float64 }

Output:

Area of main.Line is 0  
Area of main.Circle is 154  
Area of main.Triangle is 9  


#### Generics Types in GO

go doesn't support generics, but follows an impl similar to generics


<a href="L_06_Interfaces/TreeImpl.go">Generic Tree traversal</a>

#### Sorts

In [8]:
import "sort"

ints := []int{3,1,4,5}

sort.Ints(ints)

fmt.Println(ints)

[1 3 4 5]


10 <nil>

<a href="L_06_Interfaces/SortInterface.go">Sort interface</a>

#### TODO : Error interface and creating errors

#### Type assertions

NOTE : .(Goat) and .(\*Goat) are  different types and need to be asserted differently

The interface can  point to any of them

In [9]:
// Premise

type Animal interface {
	Sound() string
}

type Herbivore interface{
    Sound() string
    Herbs() string
}

type Goat struct {
	Name string
}

func (goat Goat) Sound() string {
	return "ba"
}

func (goat Goat) Herbs() string {
	return "hay"
}



In [10]:
var animal Animal = Goat{"aba"}

goat := animal.(Goat)

fmt.Println(goat.Name) // now can directly use .Name

aba


4 <nil>

In [11]:
var animalPointer Animal = &Goat{"ababa"} // The interface can point to a pointer as well

goatPointer := animalPointer.(*Goat) // The conversion here should be with a pointer

fmt.Println(goatPointer.Name)

ababa


6 <nil>

In [12]:
var animal Animal = Goat{"bababa"}

herbAnimal := animal.(Herbivore)

fmt.Println(herbAnimal.Herbs())  // Works in goland

ERROR: reflect: non-interface type passed to Type.Implements

hay